In [10]:
import tensorflow as tf
import numpy as np
from time import *
import heapq
from math import *

In [11]:
train_data = []
with open('train_data.txt','r') as infile:
    for l in infile:
        l.rstrip('\n')
        lint = [int(x) for x in l.split(',')]
        train_data.append(lint)

In [12]:
item_of_user = [[] for i in range(6040)]
for l in train_data:
    u = l[0]
    item = l[1]
    item_of_user[u].append(item)

In [13]:
test_data = []
with open('test_data.txt','r') as infile:
    for l in infile:
        l.rstrip('\n')
        lint = [int(x) for x in l.split(',')]
        test_data.append(lint)

In [14]:
negative_pair = [[] for i in range(6040)]
with open('negative_pairs.txt','r') as infile:
    for l in infile:
        l.rstrip('\n')
        u,i = (int(x) for x in l.split(','))
        negative_pair[u].append(i)

In [15]:
num_user = 6040
num_item = 3952

In [16]:
def get_data(negative_num):
    batch_user, batch_item, batch_label = [], [], []
    for u in range(num_user):
        for it in item_of_user[u]:
            batch_user.append(u)
            batch_item.append(it)
            batch_label.append(1)
            sampled_negative = 0
            while sampled_negative <negative_num:
                j = np.random.randint(num_item)
                if j in item_of_user[u]:
                    continue
                else:
                    batch_user.append(u)
                    batch_item.append(j)
                    batch_label.append(0)
                    sampled_negative += 1
    
    return np.array(batch_user), np.array(batch_item), np.array(batch_label)   

In [17]:
latent_size = 32
learning_rate = 0.001
max_epochs = 20
layers = [96, 48, 16, 8]
regularization_rate = 1e-8
batch_size = 512
negative_ratio = 4
regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)

In [18]:
tf.reset_default_graph()
embedding_u = tf.get_variable(name = 'latent_u', shape = [num_user, latent_size], initializer = tf.random_normal_initializer(stddev = 0.01))
embedding_i = tf.get_variable(name = 'latent_i', shape = [num_user, latent_size], initializer = tf.random_normal_initializer(stddev = 0.01))
fc_weights = []
fc_bias = []
for i in range(len(layers)-1):
    fc_weights.append(tf.get_variable(name = 'fc-weight' + str(i), shape = [layers[i], layers[i+1]], initializer = tf.random_normal_initializer(stddev = 0.02)))
    fc_bias.append(tf.get_variable(name = 'fc-bias' + str(i), shape = [layers[i+1]], initializer = tf.constant_initializer(0.05)))
    tf.add_to_collection("losses", regularizer(fc_weights[i]))
    
output_weight = tf.get_variable(name = 'output_weight' + str(i), shape = [layers[-1], 1], initializer = tf.random_normal_initializer(stddev = 0.01))

In [20]:
U = tf.placeholder(tf.int32, [None])
I = tf.placeholder(tf.int32, [None])
label = tf.placeholder(tf.int32, [None])

embedded_u = tf.nn.embedding_lookup(embedding_u, U)
embedded_i = tf.nn.embedding_lookup(embedding_i, I)
merged_ui = tf.concat([embedded_u, embedded_i, tf.multiply(embedded_u, embedded_i)], axis = 1)

hidden_in = merged_ui
for i in range(len(layers) - 1):
    hidden_out = tf.nn.relu(tf.matmul(hidden_in, fc_weights[i]) + fc_bias[i])
    hidden_in = hidden_out

output = tf.reshape(tf.matmul(hidden_out, output_weight), [-1])

prediction = tf.nn.sigmoid(output)
reg_term = regularizer(embedding_u) + regularizer(embedding_i) + tf.add_n(tf.get_collection('losses'))
loss = reg_term + tf.losses.log_loss(labels = label, predictions = prediction)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [22]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for ep in range(max_epochs):
        t1 = time()
        eu, ei, el = get_data(negative_ratio)
        for j in range(len(eu)//batch_size):
            sample = np.random.randint(len(eu), size = batch_size)
            bu = eu[sample]
            bi = ei[sample]
            bl = el[sample]
            loss_value, _ = sess.run([loss, optimizer], feed_dict = {U:bu, I:bi, label:bl})
            
        t2 = time()
        print("[%.1f s] After %d epochs, loss on batch is %.3f."%(t2-t1, ep, loss_value))
        
        hits = 0
        tNDCG = 0
        for u in range(num_user):
            map_item_rating = {}
            maxScore = sess.run(output, feed_dict = {U:np.array([u]), I:np.array([test_data[u][1]])})
            negative_i = negative_pair[u]
            negative_u = [u for m in range(100)]
            negative_score = sess.run(output, feed_dict = {U:negative_u, I:negative_i})
            map_item_rating[test_data[u][1]] = maxScore
            for k in range(100):
                map_item_rating[negative_i[k]] = negative_score[k]
            ranklist = heapq.nlargest(10, map_item_rating, key = map_item_rating.get)
            if test_data[u][1] in ranklist:
                hits +=1
                idx = ranklist.index(test_data[u][1])
                tNDCG += log(2)/log(idx+2)
        
        print("[%.1f s] After %d epochs, hit@10 = %.3f, NDCG@10 = %.3f."%(time()-t1, ep, hits/6040, tNDCG/6040))

[65.4 s] After 0 epochs, loss on batch is 0.335.
[68.2 s] After 0 epochs, hit@10 = 0.531, NDCG@10 = 0.467.
[65.5 s] After 1 epochs, loss on batch is 0.288.
[68.3 s] After 1 epochs, hit@10 = 0.605, NDCG@10 = 0.546.
[74.0 s] After 2 epochs, loss on batch is 0.236.
[76.9 s] After 2 epochs, hit@10 = 0.639, NDCG@10 = 0.593.
[66.6 s] After 3 epochs, loss on batch is 0.284.
[69.4 s] After 3 epochs, hit@10 = 0.654, NDCG@10 = 0.612.
[65.4 s] After 4 epochs, loss on batch is 0.255.
[71.0 s] After 4 epochs, hit@10 = 0.654, NDCG@10 = 0.605.
[66.5 s] After 5 epochs, loss on batch is 0.248.
[69.6 s] After 5 epochs, hit@10 = 0.661, NDCG@10 = 0.613.
[65.4 s] After 6 epochs, loss on batch is 0.277.
[68.2 s] After 6 epochs, hit@10 = 0.661, NDCG@10 = 0.614.
[68.2 s] After 7 epochs, loss on batch is 0.253.
[71.4 s] After 7 epochs, hit@10 = 0.666, NDCG@10 = 0.620.


KeyboardInterrupt: 